# Analyze cell-gene matrix
This Python Jupyter notebook processes the cell-gene matrix for basic purposes such as removing doublets and examining the amount of viral products per cell.

## Parameters
First, set the parameters for the notebook, such as to specify the input files and output plots.
This is done in the next cell, which is tagged as a `parameters` cell to enable [papermill parameterization](https://papermill.readthedocs.io/en/latest/usage-parameterize.html):

In [1]:
# parameters cell; in order for notebook to run this cell must define:
#  - input_matrix: filtered gene-barcode matrix from `STARsolo`
#  - input_features: list of features (genes) from `STARsolo`
#  - input_barcodes: list of cell barcodes from `STARsolo`
#  - input_viral_gtf: GTF file giving names of viral genes
#  - input_viraltag_counts: CSV file giving counts of viral variant tags, parsed by `viraltag_count` notebook
#  - input_viralbc_counts: CSV file giving counts of viral barcodes, parsed by `viraltag_count` notebook
#  - input_gapped_reads: CSV file giving info for all reads with deletions, parsed by 'gap_analysis' notebook
#  - input_gapped_reads_summary: CSV file with counts of gapped reads grouped by cell bc and gene, parsed by 'gap_analysis' notebook

## Import Python modules
We use [anndata](https://anndata.readthedocs.io/) and [scanpy](https://scanpy.readthedocs.io/) for most the data processing, and [plotnine](https://plotnine.readthedocs.io/) for ggplot2-style plotting:

In [3]:
import os

import anndata
import BCBio.GFF
from IPython.display import display, HTML
import numpy
import math
from scipy.stats import poisson
from scipy.stats import variation
import pandas as pd
from plotnine import *
import scanpy as sc

Set [scanpy](https://scanpy.readthedocs.io/) to provide lots of information including hints:

In [4]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)

Versions of key [scanpy](https://scanpy.readthedocs.io/) packages:

In [5]:
sc.logging.print_versions()

scanpy==1.4.6 anndata==0.7.1 umap==0.3.10 numpy==1.17.0 scipy==1.4.1 pandas==0.25.3 scikit-learn==0.21.3 statsmodels==0.10.1


Color-blind palette:

In [6]:
cbpalette = ('#999999', '#E69F00', '#56B4E9', '#009E73',
             '#F0E442', '#0072B2', '#D55E00', '#CC79A7')

Set [plotnine theme](https://plotnine.readthedocs.io/en/stable/api.html#themes):

In [7]:
_ = theme_set(theme_classic)

## Read the cell-gene matrix
Read the cell-gene matrix into an [AnnData](https://anndata.readthedocs.io/) annotated data object.
We can't _quite_ use the [scanpy read_10x_mtx](https://icb-scanpy.readthedocs-hosted.com/en/stable/api/scanpy.api.read_10x_mtx.html) function as the `STARsolo` output isn't in quite the right format, so instead we write our own code that accomplishes the same:

In [9]:
print(f"Reading cell-gene matrix from {input_matrix}")
adata = anndata.read_mtx(input_matrix).T

print(f"Reading features (genes) from {input_features}")
genes = pd.read_csv(input_features, header=None, sep='\t')
adata.var_names = (anndata.utils.make_index_unique(pd.Index(genes[1]))
                   .rename('gene_symbols')
                   )
adata.var['gene_ids'] = genes[0].values

print(f"Reading barcodes (cells) from {input_barcodes}")
cells = pd.read_csv(input_barcodes, header=None, sep='\t')[0]
adata.obs_names = cells.rename('cell_barcodes')

print(f"\nInfo on created annotated data object:\n{adata}")

Reading cell-gene matrix from results/aligned_fastq10x/hashing_trial3_withNH4Cl/Solo.out/Gene/filtered/matrix.mtx


FileNotFoundError: [Errno 2] No such file or directory: 'results/aligned_fastq10x/hashing_trial3_withNH4Cl/Solo.out/Gene/filtered/matrix.mtx'

## Basic analysis of total and viral UMIs per cell

### Total UMIs per cell
Annotate and plot total UMIs per cell:

In [ ]:
adata.obs['total_UMIs'] = numpy.sum(adata.X, axis=1).A1

p = (ggplot(adata.obs, aes('total_UMIs')) +
     geom_histogram(bins=50) +
     theme(figure_size=(3, 1.8)) +
     xlab('total UMIs per cell') +
     ylab('number of cells')
     )
_ = p.draw()

### Viral UMIs per cell
First get the viral transcripts:

In [ ]:
print(f"Reading names of viral transcripts from {input_viral_gtf}")
with open(input_viral_gtf) as f:
    viral_genes = [seqrecord.id for seqrecord in BCBio.GFF.parse(f)]
    
print('The viral transcripts are as follows:\n\t' + '\n\t'.join(viral_genes))

assert set(viral_genes) <= set(adata.var_names), "missing some viral genes"

Now annotate and plot total viral UMIs per cell:

In [ ]:
adata.obs['viral_UMIs'] = numpy.sum(adata[:, viral_genes].X, axis=1).A1

p = (ggplot(adata.obs, aes('viral_UMIs')) +
     geom_histogram(bins=50) +
     theme(figure_size=(3, 1.8)) +
     xlab('viral UMIs per cell') +
     ylab('number of cells')
     )
_ = p.draw()

Also annotate total cellular (non viral) UMIs per cell:

In [ ]:
adata.obs['cellular_UMIs'] = adata.obs['total_UMIs'] - adata.obs['viral_UMIs']

And annotate the fraction of UMIs that are viral:

In [ ]:
adata.obs['viral_UMI_frac'] = adata.obs['viral_UMIs'] / adata.obs['total_UMIs']

### Bulk Metrics of Viral Load
Want to know how many reads are coming from virus, rather than host, in this datset overall.

In [ ]:
bulk_viral_UMIs = adata.obs['viral_UMIs'].sum()
bulk_cellular_UMIs = adata.obs['cellular_UMIs'].sum()
bulk_total_UMIs = adata.obs['total_UMIs'].sum()
assert bulk_total_UMIs == bulk_viral_UMIs + bulk_cellular_UMIs, "Error calculating bulk metrics."

print(f"The total fraction of viral UMIs in this dataset is: {bulk_viral_UMIs/bulk_total_UMIs:.3f}")

high_virus_cells = adata.obs.query('viral_UMI_frac > 0.1')
print(f"\nThere are {len(high_virus_cells)} unfiltered cells with a high viral burden (more than 10% of UMIs come from virus).")
print(f"In these cells, the average proportion of UMIs from virus is: {high_virus_cells['viral_UMI_frac'].mean():.3f}")

### Correlation between viral and total/cellular UMIs
Below calculate the correlation between the viral and total and cellular mRNAs:

In [ ]:
for x in ['total_UMIs', 'cellular_UMIs']:
    
    corr = adata.obs[x].corr(adata.obs['viral_UMIs'], method='pearson')
    
    p = (ggplot(adata.obs, aes(x, 'viral_UMIs')) +
         geom_point(alpha=0.15) +
         theme(figure_size=(2.2, 2.2)) +
         xlab(x.replace('_', ' ')) +
         ylab('viral UMIs') +
         ggtitle(f"Correlation: {corr:.2f}")
         )
    _ = p.draw()

### Filter cells on total cellular UMIs
We filter cells that have unusually low or high number of UMIs.
We mark as `filtered` cells that fail the filter:

In [ ]:
mean_cell_UMIs = adata.obs['cellular_UMIs'].mean()
limits = (mean_cell_UMIs / 2.5, mean_cell_UMIs * 2.5)

print(f"Average of {mean_cell_UMIs:.1f} cellular UMIs / cell.\nMarking as "
      f"filtered if <{limits[0]:.1f} or >{limits[1]:.1f} cellular UMIs.")

adata.obs = (
    adata.obs
    .assign(filtered=lambda x: ((x['cellular_UMIs'] < limits[0]) |
                                (x['cellular_UMIs'] > limits[1])),
            filtered_desc=lambda x: numpy.where(x['filtered'] != True, 'retained',
                                    numpy.where(x['cellular_UMIs'] < limits[0],
                                    'too few cellular UMIs', 'too many cellular UMIs')),
            )
    )

Plot cellular and viral mRNAs in filtered versus retained cells

In [ ]:
p = (ggplot(
        adata.obs.assign(ncells=lambda x: x.groupby('filtered_desc')
                                           ['filtered']
                                           .transform('count'),
                         cell_group=lambda x: x['filtered_desc'] + ' (' + 
                                              x['ncells'].astype(str) + ' cells)'
                         ),
        aes('cellular_UMIs', 'viral_UMIs', color='cell_group')) +
     geom_point(alpha=0.15) +
     theme(figure_size=(2.2, 2.2),
           legend_title=element_blank()) +
     xlab('cellular UMIs') +
     ylab('viral UMIs') +
     scale_color_manual(values=cbpalette[1:]) +
     guides(color=guide_legend(override_aes={'alpha': 1}))
     )
_ = p.draw()

From here on out, we will restrict analyses to the non-filtered cells. A few of the plots *seem* redundant with those shown above, but those were done before filtering.

### Fraction of UMIs derived from virus in individual cells
Make a basic plot of the fraction of UMIs derived from virus:

In [ ]:
p = (ggplot(adata.obs.query('not filtered'), aes('viral_UMI_frac')) +
     geom_histogram(bins=50) +
     theme(figure_size=(3, 1.8)) +
     xlab('fraction of UMIs from virus') +
     ylab('number of cells')
     )
_ = p.draw()

### Fraction of cells infected
Want a simple estimate of what proportion of all filtered cells are infected. For now, I will simply use a cutoff of >=1% of UMIs come from viral genes.

In [ ]:
single_cells = len(adata.obs.query('(not filtered)'))
infected_cells = len(adata.obs.query('(not filtered) & (viral_UMI_frac > 0.01)'))
frac_infected = infected_cells / single_cells * 100

print(f"The total number of single cells is: {single_cells} cells.")
print(f"Of these, {infected_cells} are likely bona fide infected cells. These droplets derive at least 1% of their UMIs from viral genes.")
print(f"This is {frac_infected:.1f}% of cells in the dataset.")

I will make a plot of these cells showing the fraction of UMI from virus in each.

In [ ]:
working_df = adata.obs.query('(not filtered) & (viral_UMI_frac > 0.01)')
working_df = working_df.sort_values(by=['viral_UMI_frac'], ascending=False)
working_df['viral_UMI_frac_rank'] = working_df.reset_index().index + 1

p = (ggplot(working_df, 
            aes(x='viral_UMI_frac_rank', y='viral_UMI_frac')) +
     geom_bar(stat='identity', width=1) +
     theme(figure_size=(10, 4)) +
     xlab('viral burden rank') +
     ylab('fraction of UMIs from virus')
     )
_ = p.draw()

### Fraction of cells with high viral burden
What proportion of cells have a high proportion of viral UMIs? Say, 10% of UMIs from virus?

In [ ]:
high_virus_cells = adata.obs.query('(viral_UMI_frac > 0.1) & (not filtered)')
n_high_virus_cells = len(high_virus_cells)

print(f"There are {n_high_virus_cells} cells that passed filtering with a high viral burden (more than 10% of UMIs come from virus).")
print(f"This represents {n_high_virus_cells/infected_cells*100:.1f}% of infected cells and {n_high_virus_cells/single_cells*100:.1f}% of all cells in the datset.")
print(f"In these cells, the average proportion of UMIs from virus is: {high_virus_cells['viral_UMI_frac'].mean():.3f}")

In [ ]:
working_df = pd.DataFrame({'group':['single_cells','infected_cells','high_virus_cells'],
                           'n_cells':[f'{single_cells}',f'{infected_cells}',f'{n_high_virus_cells}'],
                           'dummy':[1,1,1]
                          })

cat_dtype = pd.api.types.CategoricalDtype(
    categories=working_df['group'], ordered=True)
working_df['group'] = working_df['group'].astype(cat_dtype)
working_df['n_cells'] = working_df['n_cells'].astype(float)
working_df['frac_cells'] = working_df['n_cells'] / float(working_df.query('group == "single_cells"')['n_cells'])
print(working_df)


p = (ggplot(working_df, 
            aes(x='dummy', y='frac_cells', fill='group')) +
     geom_bar(stat='identity', position='dodge') +
     coord_flip() +
     theme(figure_size=(4, 1.5)) +
     ylab('fraction of cells') +
     xlab('') +
     scale_fill_manual(cbpalette[0:])
     )
_ = p.draw()

## Parsing Viral Tags

### Variant infecting each cell -- unfiltered cell barcodes
I'd like to label each cell as being infected by WT or dblSyn virus. I expect some leakage of transcripts from droplet to droplet, so there may be a few reads of the opposite type. Overall, I expect most cells which are truly infected will have the vast majority of their reads from one variant or the other.

#### Loading tag counts
A lot of cell barcodes have no value for either the WT tag count or the dblSyn tag count, or both. To deal with this, **I will add replace these values with 0 right away.** This solves an error that was previously introduced, where any cell with an `NaN` value for either variant could not be processed correctly.

#### Cutoffs for purity
I will call cell as infected by either WT or dblSyn variant of virus, based on ratio of tag counts. I made arbitrary cutoffs using 0.3 and .7 as the ratios. These are at least reciprocal. I will interpret any ratio between these cutoffs is mixed.

In [ ]:
# Load counts of each viral tag from `viraltag_counts.csv`
print(f"Reading viral tag counts from {input_viraltag_counts}")
tag_counts_df = pd.read_csv(input_viraltag_counts)

tag_counts_df = tag_counts_df.rename(columns={'cell_barcode':'cell_barcodes'})

tag_counts_df = tag_counts_df.groupby(['cell_barcodes','tag_variant']).sum()
tag_counts_df = tag_counts_df.reset_index(level=['cell_barcodes','tag_variant'])
tag_counts_df = tag_counts_df.pivot_table('count', ['cell_barcodes'], 'tag_variant')
tag_counts_df = tag_counts_df.fillna(0)

In [ ]:
tag_counts_df.loc[tag_counts_df['wt'] > tag_counts_df['syn'], 'major_species'] = 'wt'
tag_counts_df.loc[tag_counts_df['wt'] < tag_counts_df['syn'], 'major_species'] = 'syn'
tag_counts_df.loc[tag_counts_df['wt'] == tag_counts_df['syn'], 'major_species'] = 'mixed'

In [ ]:
fraction_cutoff = 0.75 #Set a 25% contamination fraction as the cutoff for mixed calls.

tag_counts_df['total_tags'] = (tag_counts_df['syn'] + tag_counts_df['wt'])

tag_counts_df.loc[tag_counts_df['major_species'] == 'wt', 'major_species_frac'] = tag_counts_df['wt'] / tag_counts_df['total_tags']
tag_counts_df.loc[tag_counts_df['major_species'] == 'syn', 'major_species_frac'] = tag_counts_df['syn'] / tag_counts_df['total_tags']
tag_counts_df.loc[tag_counts_df['major_species'] == 'mixed', 'major_species_frac'] = tag_counts_df['wt'] / tag_counts_df['total_tags']

tag_counts_df['variant'] = 'mixed'

tag_counts_df.loc[tag_counts_df['major_species_frac'] > fraction_cutoff, 'variant'] = tag_counts_df['major_species']
tag_counts_df.loc[tag_counts_df['total_tags'] == 0, 'variant'] = 'no tags'
tag_counts_df = tag_counts_df.reset_index()
tag_counts_df = tag_counts_df.sort_values(by=['major_species_frac'], ascending=False)
tag_counts_df['major_species_frac_rank'] = tag_counts_df.reset_index().index + 1

In [ ]:
p = (ggplot(tag_counts_df, 
            aes(x='major_species_frac_rank', y='major_species_frac', fill='variant')) +
     geom_bar(stat='identity', width=1) +
     theme(figure_size=(8, 3)) +
     ggtitle('Fraction of viral tags from each variant\nAll cells with viral tags') +
     xlab('cell rank (least contaminating tags to most)') +
     ylab('fraction of tags from majority variant') +
     scale_fill_manual(values=cbpalette[1:])
     )
_ = p.draw()

### Variant infecting cells called as "infected"
Now, I will take the tag count and variant identity information I just calculated and merge it with the infection status information I calculated above. This will give the variant call for each "true" infected cell.

In [ ]:
adata.obs = adata.obs.merge(tag_counts_df, on='cell_barcodes', how='outer')
adata.obs

In [ ]:
single_cells = len(adata.obs.query('(not filtered)'))
    
print(f"The total number of retained single cells is: {single_cells} cells.")
print(f"I assume half of these ({single_cells/2:.0f} cells) were from the well infected",
      "with WT virus and half were from the well infected with dblSyn.\n")

for var in ['wt','syn', 'mixed', 'no tags']:
    var_infected_cells = len(adata.obs.query('(not filtered) & (viral_UMI_frac > 0.01) & (variant == @var)'))
    var_frac_infected = var_infected_cells / (single_cells / 2) * 100

    if var == 'wt' or var == 'syn':
        print(f"There were {var_infected_cells} cells called as infected with {var} virus.")
        print(f"Assuming half the total single cells came from this sample well, {var_frac_infected:.0f}% of the cells in that well were infected.\n")
    else:
        print(f"There were {var_infected_cells} droplets called as {var}.")
    

Plot the proportions of tag counts for each true infected cell.

In [ ]:
working_df = adata.obs.query('(not filtered) & (viral_UMI_frac > 0.01)')
working_df = working_df.sort_values(by=['major_species_frac'], ascending=False)
working_df['major_species_frac_rank'] = working_df.reset_index().index + 1
p = (ggplot(working_df, 
            aes(x='major_species_frac_rank', y='major_species_frac', fill='variant')) +
     geom_bar(stat='identity', width=1) +
     theme(figure_size=(8, 3)) +
     ggtitle('Fraction of viral tags from each variant\ninfected cells with viral tags') +
     xlab('cell rank (least contaminating tags to most)') +
     ylab('fraction of tags from majority variant') +
     scale_fill_manual(values=cbpalette[1:])
     )
_ = p.draw()

## Parsing viral barcodes
I would like to know how many virions infected each cell in this dataset.

#### Loading barcodes
The barcodes are parsed in the `count_viralbc_fastq10x` notebook. This, in turn, calls the `extract_tags` functions to identify reads that align and cover the position of the viral barcode in the reference genome. The barcodes are checked for quality and then exported.

I will import the barcode counts from the `viralbc_counts` CSV. This file lists the counts for each cell/segment/viral barcode combination.

#### Counting viral barcodes per cell -- unfiltered cell barcodes
For each cell, I will count the number of "true" viral barcodes that are present for each segment. I have a few naiive assumptions:
* If capture is good for both segments, I expect the number of barcodes on HA to (usually) match the number of barcodes on NA.
* At a rate of ~20% of cells infected, assuming poisson occupancy, I expect ~18% of cells to have only 1 viral barcode, and about 2% of cells to have >1 viral barcodes.
* These cells were infected in a large dish with a large viral innoculum. With the diversity of the viral barcodes in the viral stock, it is likely that the same barcode will show up in multiple cells.

I will define "true" barcodes as barcodes that are seen in more than X UMI for a given cell/segment combination. The counts for these data are generally quite low right now. Jesse shared the advice that at low counts, using integers of counts is a better method than percentages. This makes sense given that a single count is a large percentage of these data. To see how filtering varies with cutoff choice, I will test a few different cutoffs and plot them side by side.

In [ ]:
# Load counts of each viral barcode from `viraltag_counts.csv`
print(f"Reading viral barcode counts from {input_viralbc_counts}")
bc_counts_df = pd.read_csv(input_viralbc_counts)

bc_counts_df = bc_counts_df.rename(columns={'cell_barcode':'cell_barcodes'})

print(bc_counts_df.head())
#print(bc_counts_df.size)

First, I will filter this data by a variety of cutoffs. I will append each filter result to the dataframe.

In [ ]:
cutoffs = [2,3,5,10]

working_df = bc_counts_df.copy(deep=True)
for cutoff in cutoffs:

    print(f"Marking as filtered if less than {cutoff:.0f} UMIs for a given barcode in a cell.")

    working_df['cutoff'] = cutoff
    working_df['filtered'] = working_df['count'] < cutoff
    #print(working_df.head())
    
    bc_counts_df = bc_counts_df.append(working_df)
    
bc_counts_df['cutoff'] = bc_counts_df['cutoff'].fillna(0)
bc_counts_df['filtered'] = bc_counts_df['filtered'].fillna(False)

bc_counts_df = bc_counts_df.reset_index()
bc_counts_df = bc_counts_df.drop(labels=['index'],axis=1)

#print(bc_counts_df.size)

In [ ]:
bc_counts_df.head()

In [ ]:
p = (ggplot(bc_counts_df, aes('cutoff', color='filtered')) +
     geom_point(stat='count') +
     geom_line(stat='count') +
     facet_wrap('~gene') +
     theme(figure_size=(6, 3),
           axis_text_x=element_text(angle=90)) +
     scale_y_continuous() +
     ylab('viral barcodes\npassing filter')
     )

_ = p.draw()

Then, I will count the number of unique viral barcodes associated with each cell barcode-gene combination:

In [ ]:
viralbc_per_cell_df = bc_counts_df[bc_counts_df['filtered'] == False].groupby(['cell_barcodes','gene','cutoff']).count()
viralbc_per_cell_df =  viralbc_per_cell_df.reset_index(level=['cell_barcodes','gene','cutoff'])
viralbc_per_cell_df = viralbc_per_cell_df.drop(labels=['count','filtered'],axis=1)
viralbc_per_cell_df = viralbc_per_cell_df.rename(columns={'viral_barcode':'n_viralbc'})
viralbc_per_cell_df['n_viralbc'] = viralbc_per_cell_df['n_viralbc'].astype(int)
viralbc_per_cell_df.head()

Then, I'll examine the distributions of cell barcodes per cell. Want to find a cutoff range that is fairly stable and does not change the distribtuion much.

In [ ]:
p = (ggplot(viralbc_per_cell_df, aes('n_viralbc')) +
     geom_histogram(binwidth=1) +
     facet_grid('gene~cutoff') +
     scale_x_continuous(breaks=list(range(0,36)), limits = [-0.5,10.5]) +
     theme(figure_size=(9, 4),
           axis_text_x=element_text(angle=90)) +
     ylab('number of cells') +
     xlab('number of viral barcodes')
     )

_ = p.draw()

In [ ]:
max(viralbc_per_cell_df['n_viralbc'])

ggplot throws an error for "non-finite values". This is because some cells are counted as having up to 36 viral barcodes, and that is outside the limits of the plot. That is fine -- they are probably not real, and there are so few cells in this range that they cannot be visualized anyway.

### Choosing an empirical cutoff for viral barcode UMI counts
Based on the plots above, filtering is somewhat stable using a cutoff in the range `3-10`. No filtering produces a lot of cell which are called as having `>2` virions infecting. The number of filtered barcodes changes rather dramatically by changing the cutoff from `2` to `3`, but the number of virions infecting each cell histogram does not change much. Converseley, the number of filtered barcodes does not change much  from `3` to `10`.

**For this first pass, I will choose a cutoff of `3`. It is an intermediate filtering plot and histogram that looks pretty similar to 5, but yields more total cells and more co-infected cells.**

In [ ]:
cutoff_choice = 3

viralbc_per_cell_df = viralbc_per_cell_df[viralbc_per_cell_df['cutoff'] == cutoff_choice]
#viralbc_per_cell_df.head()

In [ ]:
viralbc_per_cell_df = viralbc_per_cell_df.pivot_table(index='cell_barcodes', columns='gene', values='n_viralbc')
viralbc_per_cell_df = viralbc_per_cell_df.rename(columns={'fluHA':'ha_n_viralbc', 'fluNA':'na_n_viralbc'})
viralbc_per_cell_df.head()

### Viral barcode count for cells called as infected
Now, I will take the viral barcode count information I just calculated and merge it with the infection status information I calculated above. This will give the variant call for each "true" infected cell.

**N.b.** Many cells have `NaN` values for one (or both) of the barcoded segments. These represent cells where a barcode was either not picked up or did not pass the filtering above. I could fill these in with `0`, which would make downstream plotting easier. However, putting a value on this is a bit less representative, so for now, I will leave them as `NaN`.

In [ ]:
adata.obs = adata.obs.merge(viralbc_per_cell_df, on='cell_barcodes', how='outer')
adata.obs.query('(not filtered) & (viral_UMI_frac > 0.01)').head()

In [ ]:
max_ha_n_viralbc = int(adata.obs.query('(not filtered)')['ha_n_viralbc'].max())
    
print(f"The total number of retained single cells is: {single_cells} cells.")
print(f"The total number of infected  cells is: {infected_cells} cells.")
print(f"The fraction of infected  cells is: {frac_infected:.1f}% of cells.")
                       
print(f"\nThe highest number of viral barcodes found in a true infected cell is {max_ha_n_viralbc} viral barcodes.\n")


# cells_with_x_virions = len(adata.obs.query('(not filtered) & (viral_UMI_frac > 0.01) & (ha_n_viralbc.isnull())', engine='python'))
# frac_x_virions = cells_with_x_virions / (single_cells) * 100
# print(f"There are {cells_with_x_virions} true infected cells that could not have any viral barcodes called.")
# print(f"This is {frac_x_virions:.1f}% of all cells in the dataset.\n")

# for x in range(1, max_ha_n_viralbc+1):
#     cells_with_x_virions = len(adata.obs.query(f'(not filtered) & (viral_UMI_frac > 0.01) & (ha_n_viralbc == {x})'))
#     frac_x_virions = cells_with_x_virions / (single_cells) * 100
#     print(f"There are {cells_with_x_virions} true infected cells with {x} viral barcodes present in the HA segment of that cell.")
#     print(f"This is {frac_x_virions:.1f}% of all (infected and unifected) cells in the dataset.\n")
    

#### Plot viral barcode distributions in infected cells
Now I would like to plot the viral barcodes a few ways.
* First, I would like to compare HA and NA viral barcode distributions.
* Second, I would like to compare HA and NA viral barcode counts within each cell.
* Third, I would like to compare the number of distribution of viral barcodes in these data to the expectation of viral barcode counts under an assumption of poisson occupancy.



In [ ]:
# I should really melt this dataframe so that I can facet this plot, rather than doing it twice.
# The melt syntax is tripping me up, so for now I'm just going to do it by hand.

working_df = adata.obs.query('(not filtered) & (viral_UMI_frac > 0.01)')
working_df['ha_n_viralbc'] = working_df['ha_n_viralbc'].fillna(0)
working_df['na_n_viralbc'] = working_df['na_n_viralbc'].fillna(0)

p = (ggplot(working_df, aes('ha_n_viralbc', fill='variant')) +
     geom_histogram(binwidth=1) +
     scale_x_continuous(breaks=list(range(0,5)), limits=[-.5,4.5]) +
     theme(figure_size=(3, 2),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Number of viral barcodes call on HA segment\nof true infected cells') +
     ylab('number of cells') +
     xlab('number of viral barcodes') +
     scale_fill_manual(values=cbpalette[1:])
     )

_ = p.draw()

In [ ]:
p = (ggplot(working_df, aes('na_n_viralbc', fill='variant')) +
     geom_histogram(binwidth=1) +
     scale_x_continuous(breaks=list(range(0,5)), limits=[-.5,4.5]) +
     theme(figure_size=(3, 2),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Number of viral barcodes call on NA segment\nof true infected cells') +
     ylab('number of cells') +
     xlab('number of viral barcodes') +
     scale_fill_manual(values=cbpalette[1:])
     )

_ = p.draw()

In [ ]:
p = (ggplot(working_df, aes(x='ha_n_viralbc', y='na_n_viralbc', color='variant')) +
     geom_jitter(width=0.15, height=0.15) +
     theme(figure_size=(5, 3.5),
           axis_text_x=element_text(angle=90)) +
     scale_x_continuous(breaks=list(range(0,5)), limits=[-.5,4.5]) +
     scale_y_continuous(breaks=list(range(0,5)), limits=[-.5,4.5]) +
     ggtitle('Number of viral barcodes called\nin true infected cells') +
     xlab('number of viral barcodes on HA segment') +
     ylab('number of viral barcodes on NA segment') +
     scale_color_manual(values=cbpalette[1:])
     )

_ = p.draw()

### How are viral barcodes shared across cells?
I see that most cells have 0 or 1 viral barcodes. Are most viral barcodes only found in 1 cell?

#### Viral barcode sharing in all cells -- unfiltered
First, I will calculate the distribution for the raw data -- all barcode counts, filtered by various cutoffs.

In [ ]:
cells_per_viralbc_df = bc_counts_df[bc_counts_df['filtered'] == False].groupby(['viral_barcode','gene','cutoff']).count()
cells_per_viralbc_df =  cells_per_viralbc_df.reset_index(level=['viral_barcode','gene','cutoff'])
cells_per_viralbc_df = cells_per_viralbc_df.drop(labels=['count','filtered'],axis=1)
cells_per_viralbc_df = cells_per_viralbc_df.rename(columns={'cell_barcodes':'n_cells'})
cells_per_viralbc_df['n_cells'] = cells_per_viralbc_df['n_cells'].astype(int)
cells_per_viralbc_df.head()

In [ ]:
#What is the maximum number of cells a viral barcode is found in (Raw Counts Data)?
cells_per_viralbc_df['n_cells'].max()

In [ ]:
p = (ggplot(cells_per_viralbc_df, aes('n_cells')) +
     geom_histogram(binwidth=1) +
     facet_grid('gene~cutoff') +
     scale_x_continuous(breaks=list(range(0,36)), limits = [-0.5,10.5]) +
     theme(figure_size=(9, 4),
           axis_text_x=element_text(angle=90)) +
     ylab('number of viral barcodes') +
     xlab('found in number of cells')
     )

_ = p.draw()

#### Viral barcode sharing in true infected cells
Next, I will add these data to the adata.obs dataframe and see how many of the infected cells I am analyzing in this dataset share viral barcodes with another cell.

In [ ]:
#Merge raw bc counts, as I need cell barcode information, viral barcode information, and gene information.
#These are note present in above summary dataframes.

working_df = bc_counts_df.copy()
working_df = working_df.query(f'(not filtered) & (cutoff == {cutoff_choice})')
working_df = working_df.drop(labels=['count','cutoff','filtered'], axis=1)

working_df = working_df.merge(adata.obs, on='cell_barcodes', how='outer')
working_df = working_df.query('(not filtered) & (viral_UMI_frac > 0.01)')

working_df.head()                 

In [ ]:
#Get counts of how many cells are associated with each viral barcode/gene combination.

working_df = working_df.reset_index()
working_df = working_df.drop(labels='index', axis=1)
working_df = working_df.groupby(['viral_barcode','gene']).count()
working_df = working_df.reset_index()
working_df = working_df[['viral_barcode','gene', 'cell_barcodes']]
working_df = working_df.rename(columns = {'cell_barcodes':'n_cells'})
#working_df.head()

In [ ]:
# Plot the results.

p = (ggplot(working_df, aes('n_cells')) +
     geom_histogram(binwidth=1) +
     facet_grid('~gene') +
     scale_x_continuous(breaks=list(range(0,36)), limits = [-0.5,10.5]) +
     theme(figure_size=(7, 2),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Viral barcode sharing across infected cells') +
     ylab('number of viral barcodes') +
     xlab('found in number of cells')
     )

_ = p.draw()

### Single-virion infections and co-infected cells
I will  label cells as uninfected, infected by 1 virion, or infected by more than one virion. My definition for each category is as follows:  
* uninfected (<1% of UMIs from viral reads)
* infected by 1 virion (maximum of 1 viral barcode, when cosidering **both** HA and NA segments)
* infected by >1 virion (>1 viral barcode on **either** HA or NA segment).

Then, I will plot some simple summary statistics to see how they vary by group.

#### Count virions and label cells

In [ ]:
def count_virions(row):
    #print(row['ha_n_viralbc'], row['na_n_viralbc'])
    if numpy.isnan(row['ha_n_viralbc']) and numpy.isnan(row['na_n_viralbc']):
        return 0
    elif row['ha_n_viralbc'] > 1 or row['na_n_viralbc'] > 1:
        return max(row['ha_n_viralbc'],row['na_n_viralbc'])
    else:
        return 1

adata.obs['n_virions'] = adata.obs.apply(lambda row: count_virions(row), axis=1)

In [ ]:
p = (ggplot(adata.obs.query('(not filtered) and (viral_UMI_frac > 0.01)'), aes('n_virions', fill='variant')) +
     geom_histogram(binwidth=1) +
     scale_x_continuous(breaks=list(range(0,6)), limits=[-.5,5.5]) +
     theme(figure_size=(4, 2.5),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Number virions infecting\ncells with >1% UMI from virus') +
     ylab('number of cells') +
     xlab('number of virions in cell') +
     scale_fill_manual(values=cbpalette[1:])
     )

_ = p.draw()

In [ ]:
max(adata.obs.query('not filtered')['n_virions'])

Once again, there is an error thrown because a few cells with high numbers of virions are cut off on this plot.

#### Do viral barcode distributions fit a Poisson model?
I'd like to compare the distribution of barcode counts to a null of model. Assuming poisson occupancy of virus in cells, given the overall infection rate, how many cells would we expect to have 0 viral barcodes, 1 viral barcodes, 2 viral barcodes, etc...

**Calculating mu from observed number of infected cells**

Our `frac_infected` variable gives the number of cells that we have called as infected. This includes cells that were infected by a single virion `(k=1)`, as well as cells that were infected by any other number of virions `(k>=1)`. At low MOI, this value is very close to the average MOI (called `poisson_u` in my code below). However, at higher MOI, these values deviate substantially, as more and more cells are infected by multiple virions.

To calculate the MOI, or average number of virions per cell, of the observed data, I will work backwards from the poisson formula. 

```
P(k=poisson_k|poisson_u) = (numpy.e**(-1*poisson_u)) * (poisson_u**poisson_k) / math.factorial(poisson_k)

```

Thankfully, working with a distinction between 0 and 1 simplifies the math quite a bit:

```
P(k>=1|poisson_u) = frac_infected
P(k=0|poisson_u) = 1 - P(k>=1|poisson_u)
frac_not_infected = 1 - frac_infected

frac_not_infected = P(k=0|poisson_u) = (numpy.e**(-1*poisson_u)) * (poisson_u**0) / math.factorial(0)

(poisson_u**0) = 1
math.factorial(0) = 1

frac_not_infected = numpy.e**(-1*poisson_u)
math.log(frac_not_infected) = -1*poisson_u
poisson_u = -1 * math.log(frac_not_infected).
```

The average of the poisson distribution (or MOI) is simply the `-log` of `1 - the fraction infected` (observed).

In [ ]:
# Calculate poisson expectation
poisson_expectation = list()

for k in range(0, max_ha_n_viralbc+1):

    poisson_k = k
    frac_not_infected = 1 - frac_infected/100
    poisson_u = -1 * math.log(frac_not_infected)
            
#     print("Calculating poisson probabilities of viral barcode counts with:\n",
#          f"Poisson random variable X = {poisson_k}\n",
#          f"Infection rate = {frac_infected/100:.3f};\t",
#          f"Poisson u = {poisson_u:0.3f}")

    r = poisson.pmf(k=poisson_k, mu=poisson_u)
#     print(f" The expected proportion of cells with {poisson_k} viral barcodes is {r:.2f}.")
#     print(f" This is {r*single_cells:.0f} of all {single_cells} cells in the dataset.\n")
    
    poisson_expectation.append({'n_viralbc':poisson_k,
                                'n_cells': int(r*single_cells),
                               })

poisson_expectation = pd.DataFrame(poisson_expectation)
poisson_expectation['source'] = 'poisson model'

In [ ]:
# p = (ggplot(poisson_expectation, aes(x='n_viralbc', y='n_cells')) +
#      geom_bar(stat='identity') +
#      scale_x_continuous(limits=[-0.5,4.5]) +
#      theme(figure_size=(5, 3.5),
#            axis_text_x=element_text(angle=90)) +
#      ggtitle('Numnber of viral barcodes expected\nunder Poisson model') +
#      ylab('number of cells') +
#      xlab('number of viral barcodes') +
#      annotate('text', x=4, y= 2800, label=f'MOI = {poisson_u:.3f}')
#      )

# _ = p.draw()

In [ ]:
#Compare to observed results
working_df = pd.DataFrame()
working_df['n_cells'] = adata.obs.query('(not filtered)')['n_virions'].value_counts(dropna=False)
working_df['source'] = 'observed'
    
working_df = working_df.reset_index().rename(columns={'index': 'n_viralbc'})

working_df = working_df.append(poisson_expectation)
working_df = working_df.fillna(0)

#working_df

Finally, plot the observed distribuiont of viral barcodes per cell alongside the poisson expectation, given the fraction of cells called as infected.

In [ ]:
p = (ggplot(working_df, aes(x='n_viralbc', y='n_cells', fill='source')) +
     geom_bar(stat='identity', position='dodge') +
     scale_x_continuous(limits=[-0.5,4.5]) +
     theme(figure_size=(5, 3.5),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Number of viral barcodes observed vs.\n number expected under Poisson model') +
     ylab('number of cells') +
     xlab('number of viral barcodes') +
     annotate('text', x=4, y= 2800, label=f'MOI = {poisson_u:.3f}') +
     scale_fill_manual(values=['#20a0c5','#444444'])
     )

_ = p.draw()

In [ ]:
def cell_infection_status(row):
    #print(row['ha_n_viralbc'], row['na_n_viralbc'])
    if row['viral_UMI_frac'] <= 0.01:
        return 'uninfected'
    else:
        if row['n_virions'] == 0:
            return 'no viral bc detected'
        elif row['n_virions'] == 1:
            return 'single virion'
        else:
            return 'co-infected'

adata.obs['infection_status'] = adata.obs.apply(lambda row: cell_infection_status(row), axis=1)
adata.obs['infection_status'] = pd.Categorical(adata.obs['infection_status'], categories = ['uninfected','single virion','co-infected','no viral bc detected'], ordered = True)

In [ ]:
#### Number of cells called as singly-infected or co-infected

In [ ]:
p = (ggplot(adata.obs.query('(not filtered) and (infection_status != "uninfected")'), aes('infection_status', fill='variant')) +
     geom_bar(stat='count') +
     theme(figure_size=(4, 2.5),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Number of cells infected by single virion or co-infected') +
     ylab('number of cells') +
     xlab('number of virions in cell') +
     scale_fill_manual(values=cbpalette[1:])
     )

_ = p.draw()

In [ ]:
#### Total UMIs by infection status

In [ ]:
p = (ggplot(adata.obs.query('(not filtered) and (infection_status != "uninfected")'), aes(x='infection_status', y='total_UMIs', fill='infection_status')) +
     geom_boxplot() +
     theme(figure_size=(4, 2.5),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Total UMIs by infection status') +
     ylab('number of UMIs per cell') +
     xlab('infection status') +
     scale_fill_manual(values=cbpalette[0:])
     )

_ = p.draw()

In [ ]:
#### Number of viral UMIs by infection status

In [ ]:
p = (ggplot(adata.obs.query('(not filtered) and (infection_status != "uninfected")'), aes(x='infection_status', y='viral_UMIs', fill='infection_status')) +
     geom_boxplot() +
     theme(figure_size=(4, 2.5),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Viral UMIs by infection status') +
     ylab('number of UMIs per cell') +
     xlab('infection status') +
     scale_fill_manual(values=cbpalette[0:])
     )

_ = p.draw()

#### Fraction of viral UMIs by infection status

In [ ]:
p = (ggplot(adata.obs.query('(not filtered) and (infection_status != "uninfected")'), aes(x='infection_status', y='viral_UMI_frac', fill='infection_status')) +
     geom_boxplot() +
     theme(figure_size=(4, 2.5),
           axis_text_x=element_text(angle=90)) +
     ggtitle('Fraction of UMIs from virus\nby infection status') +
     ylab('fraction of UMIs from virus') +
     xlab('infection status') +
     scale_fill_manual(values=cbpalette[0:])
     )

_ = p.draw()

#### Distribution of viral UMI fraction by infection status

In [ ]:
working_df = adata.obs.query('(not filtered) & (viral_UMI_frac > 0.01)')
working_df = working_df.sort_values(by=['viral_UMI_frac'], ascending=False)
working_df['viral_UMI_frac_rank'] = working_df.reset_index().index + 1

p = (ggplot(working_df, 
            aes('viral_UMI_frac', fill='infection_status')) +
     geom_histogram(binwidth=0.02) +
     theme(figure_size=(6, 3)) +
     xlab('fraction of UMIs from virus') +
     ylab('number of cells') +
     scale_fill_manual(cbpalette[1:])
     )
_ = p.draw()

#### Coefficient of variation by infection status
The coefficient of variation is calculated as the standard deviation divided by the mean.

In [ ]:
working_df.groupby('infection_status')['viral_UMI_frac'].std() / working_df.groupby('infection_status')['viral_UMI_frac'].mean()

#### Fraction viral UMI by number of infecting virions

In [ ]:
p = (ggplot(adata.obs.query('(not filtered) and (infection_status != "uninfected")'), aes(x='n_virions', y='viral_UMI_frac', fill='infection_status')) +
     geom_jitter(width=0.3, alpha=0.7) +
     theme(figure_size=(3, 2.5),
           axis_text_x=element_text(angle=90)) +
     scale_x_continuous(breaks=list(range(0,6)), limits = [-0.5,5.5]) +
     ggtitle('Fraction of UMIs from virus\nby number of infecting virions') +
     ylab('fraction of UMIs from virus') +
     xlab('number of virions infecting cell') +
     scale_fill_manual(values=cbpalette[0:])
     )

_ = p.draw()

#### Viral gene expression

How does expression of viral genes compare across cells that are singly-infected or co-infected?

In [ ]:
adata.obs_names = adata.obs_names.astype(str) #Convert obsnames to string type for subsetting

In [ ]:
#Let's make a copy of the adata object that only includes cells that werent' filtered out.
#This will be easier to work with.
filtered_cells = adata[adata.obs['filtered']==False, :]

In [ ]:
sc.pl.stacked_violin(filtered_cells, viral_genes, groupby='infection_status', figsize=[10,8])

#### Clustering cells by transcriptomes

In [ ]:
sc.pp.neighbors(filtered_cells)
sc.tl.umap(filtered_cells)

sc.set_figure_params(figsize=[10,8])

In [ ]:
sc.pl.umap(filtered_cells, color=['viral_UMI_frac'],
           title=['Cells clustered by PCA\nand colored by fraction viral UMIs'],
          color_map='tab10')

In [ ]:
sc.pl.umap(filtered_cells, color=['infection_status'],
           title=['Cells clustered by PCA\nand colored by infection_status'],
          palette=cbpalette[0:])